In [49]:
from pathlib import Path

import polars as pl
from polars import col
import numpy as np

DATA_DIR = Path("../../data/")

### [176. Second Highest Salary](https://leetcode.com/problems/second-highest-salary/)

In [2]:
dfs=[pl.read_csv(DATA_DIR / file) for file in ["176_1.csv", "176_2.csv"]]
print(*dfs)

shape: (3, 2)
┌─────┬────────┐
│ id  ┆ salary │
│ --- ┆ ---    │
│ i64 ┆ i64    │
╞═════╪════════╡
│ 1   ┆ 100    │
│ 2   ┆ 200    │
│ 3   ┆ 300    │
└─────┴────────┘ shape: (1, 2)
┌─────┬────────┐
│ id  ┆ salary │
│ --- ┆ ---    │
│ i64 ┆ i64    │
╞═════╪════════╡
│ 1   ┆ 100    │
└─────┴────────┘


MySQL
```mysql
select max(salary) as SecondHighestSalary
from Employee
where salary < (select max(salary) from Employee)
```

In [3]:
def lc_176(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df.filter(pl.col("salary") < pl.max("salary"))
        .select(pl.max("salary").alias("SecondHighestSalary"))
    )
    
print(*[lc_176(df) for df in dfs])

shape: (1, 1)
┌─────────────────────┐
│ SecondHighestSalary │
│ ---                 │
│ i64                 │
╞═════════════════════╡
│ 200                 │
└─────────────────────┘ shape: (1, 1)
┌─────────────────────┐
│ SecondHighestSalary │
│ ---                 │
│ i64                 │
╞═════════════════════╡
│ null                │
└─────────────────────┘


### [180. Consecutive Numbers](https://leetcode.com/problems/consecutive-numbers/)

In [54]:
print(df := pl.read_csv(DATA_DIR / "180.csv"))

shape: (7, 2)
┌─────┬─────┐
│ id  ┆ num │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 1   ┆ 1   │
│ 2   ┆ 1   │
│ 3   ┆ 1   │
│ 4   ┆ 2   │
│ 5   ┆ 1   │
│ 6   ┆ 2   │
│ 7   ┆ 2   │
└─────┴─────┘


MySQL
```mysql
with t as (
    select id, num as num1, 
        lead(num) over w as num2,
        lead(num, 2) over w as num3
    from Logs
    window w as (order by id))
select distinct num1 as ConsecutiveNums
from t
where num1 = num2 and num2 = num3
```

In [68]:
print(
    df.sort("id")
    .filter(col("num")==col("num").shift(1), col("num")==col("num").shift(2))
    .select(col("num").unique().alias("ConsecutiveNums"))
)

shape: (1, 1)
┌─────────────────┐
│ ConsecutiveNums │
│ ---             │
│ i64             │
╞═════════════════╡
│ 1               │
└─────────────────┘


### [184. Department Highest Salary](https://leetcode.com/problems/department-highest-salary/)

In [4]:
df_department = pl.read_csv(DATA_DIR / "184_department.csv")
df_employee = pl.read_csv(DATA_DIR / "184_employee.csv")
print(df_employee, df_department)

shape: (5, 4)
┌─────┬───────┬────────┬──────────────┐
│ id  ┆ name  ┆ salary ┆ departmentId │
│ --- ┆ ---   ┆ ---    ┆ ---          │
│ i64 ┆ str   ┆ i64    ┆ i64          │
╞═════╪═══════╪════════╪══════════════╡
│ 1   ┆ Joe   ┆ 70000  ┆ 1            │
│ 2   ┆ Jim   ┆ 90000  ┆ 1            │
│ 3   ┆ Henry ┆ 80000  ┆ 2            │
│ 4   ┆ Sam   ┆ 60000  ┆ 2            │
│ 5   ┆ Max   ┆ 90000  ┆ 1            │
└─────┴───────┴────────┴──────────────┘ shape: (2, 2)
┌─────┬───────┐
│ id  ┆ name  │
│ --- ┆ ---   │
│ i64 ┆ str   │
╞═════╪═══════╡
│ 1   ┆ IT    │
│ 2   ┆ Sales │
└─────┴───────┘


MySQL
```mysql
with e as (
    select *, rank() over (partition by departmentId order by salary desc) as r
    from Employee)
select d.name as "Department", e.name as "Employee", e.salary as "Salary"
from e inner join Department as d
    on e.departmentId = d.id
where r = 1
```

In [5]:
print(
    df_employee.filter(salary=pl.col("salary").max().over("departmentId"))
    .join(df_department, left_on="departmentId", right_on="id")
    .select(Department="name_right", Employee="name", Salary="salary")
)

shape: (3, 3)
┌────────────┬──────────┬────────┐
│ Department ┆ Employee ┆ Salary │
│ ---        ┆ ---      ┆ ---    │
│ str        ┆ str      ┆ i64    │
╞════════════╪══════════╪════════╡
│ IT         ┆ Jim      ┆ 90000  │
│ Sales      ┆ Henry    ┆ 80000  │
│ IT         ┆ Max      ┆ 90000  │
└────────────┴──────────┴────────┘


### [185. Department Top Three Salaries](https://leetcode.com/problems/department-top-three-salaries/)

In [6]:
df_department = pl.read_csv(DATA_DIR / "185_department.csv")
df_employee = pl.read_csv(DATA_DIR / "185_employee.csv")
print("department:", df_department)
print("employee:", df_employee)

department: shape: (2, 2)
┌─────┬───────┐
│ id  ┆ name  │
│ --- ┆ ---   │
│ i64 ┆ str   │
╞═════╪═══════╡
│ 1   ┆ IT    │
│ 2   ┆ Sales │
└─────┴───────┘
employee: shape: (7, 4)
┌─────┬───────┬────────┬──────────────┐
│ id  ┆ name  ┆ salary ┆ departmentId │
│ --- ┆ ---   ┆ ---    ┆ ---          │
│ i64 ┆ str   ┆ i64    ┆ i64          │
╞═════╪═══════╪════════╪══════════════╡
│ 1   ┆ Joe   ┆ 85000  ┆ 1            │
│ 2   ┆ Henry ┆ 80000  ┆ 2            │
│ 3   ┆ Sam   ┆ 60000  ┆ 2            │
│ 4   ┆ Max   ┆ 90000  ┆ 1            │
│ 5   ┆ Janet ┆ 69000  ┆ 1            │
│ 6   ┆ Randy ┆ 85000  ┆ 1            │
│ 7   ┆ Will  ┆ 70000  ┆ 1            │
└─────┴───────┴────────┴──────────────┘


MySQL
```mysql
with e as (
    select *, dense_rank() over (partition by departmentId order by salary desc) as r
    from Employee)
select 
    d.name as Department,
    e.name as Employee,
    e.salary as Salary
from e inner join Department as d
on e.departmentId = d.id
where r <= 3
```

In [7]:
print(
    df_employee.filter(pl.col("salary").rank("dense", descending=True).over("departmentId") <= 3)
    .join(df_department, left_on="departmentId", right_on="id")
    .select(Department="name_right", Employee="name", Salary="salary")
)

shape: (6, 3)
┌────────────┬──────────┬────────┐
│ Department ┆ Employee ┆ Salary │
│ ---        ┆ ---      ┆ ---    │
│ str        ┆ str      ┆ i64    │
╞════════════╪══════════╪════════╡
│ IT         ┆ Joe      ┆ 85000  │
│ Sales      ┆ Henry    ┆ 80000  │
│ Sales      ┆ Sam      ┆ 60000  │
│ IT         ┆ Max      ┆ 90000  │
│ IT         ┆ Randy    ┆ 85000  │
│ IT         ┆ Will     ┆ 70000  │
└────────────┴──────────┴────────┘


### [262. Trips and Users](https://leetcode.com/problems/trips-and-users/)

In [9]:
df_trips = pl.read_csv(DATA_DIR / "262_trips.csv", dtypes={"request_at": pl.Date})
df_users = pl.read_csv(DATA_DIR / "262_users.csv")
print("trips:", df_trips)
print("users:", df_users)

trips: shape: (10, 6)
┌─────┬───────────┬───────────┬─────────┬─────────────────────┬────────────┐
│ id  ┆ client_id ┆ driver_id ┆ city_id ┆ status              ┆ request_at │
│ --- ┆ ---       ┆ ---       ┆ ---     ┆ ---                 ┆ ---        │
│ i64 ┆ i64       ┆ i64       ┆ i64     ┆ str                 ┆ date       │
╞═════╪═══════════╪═══════════╪═════════╪═════════════════════╪════════════╡
│ 1   ┆ 1         ┆ 10        ┆ 1       ┆ completed           ┆ 2013-10-01 │
│ 2   ┆ 2         ┆ 11        ┆ 1       ┆ cancelled_by_driver ┆ 2013-10-01 │
│ 3   ┆ 3         ┆ 12        ┆ 6       ┆ completed           ┆ 2013-10-01 │
│ 4   ┆ 4         ┆ 13        ┆ 6       ┆ cancelled_by_client ┆ 2013-10-01 │
│ …   ┆ …         ┆ …         ┆ …       ┆ …                   ┆ …          │
│ 7   ┆ 3         ┆ 12        ┆ 6       ┆ completed           ┆ 2013-10-02 │
│ 8   ┆ 2         ┆ 12        ┆ 12      ┆ completed           ┆ 2013-10-03 │
│ 9   ┆ 3         ┆ 10        ┆ 12      ┆ completed   

MySQL
```mysql
select request_at as "Day", 
    round(avg(status != 'completed'), 2) as "Cancellation Rate"
from Trips t
where request_at between '2013-10-01' and '2013-10-03'
    and not exists (
        select 1 from Users
        where banned = 'Yes' and users_id in (t.client_id, t.driver_id)
    )
group by request_at
```

In [30]:
print(
    df_trips.filter(
        ~pl.col("driver_id").is_in(banned:=df_users.filter(banned="Yes").get_column("users_id")),
        ~pl.col("client_id").is_in(banned),
        pl.col("request_at").is_between(pl.date(2013, 10, 1), pl.date(2013, 10, 3)))
    .group_by(pl.col("request_at").alias("Day"))
    .agg((pl.col("status") != "completed").mean().round(2).alias("Cancellation Rate"))
)

shape: (3, 2)
┌────────────┬───────────────────┐
│ Day        ┆ Cancellation Rate │
│ ---        ┆ ---               │
│ date       ┆ f64               │
╞════════════╪═══════════════════╡
│ 2013-10-03 ┆ 0.5               │
│ 2013-10-02 ┆ 0.0               │
│ 2013-10-01 ┆ 0.33              │
└────────────┴───────────────────┘


### [601. Human Traffic of Stadium](https://leetcode.com/problems/human-traffic-of-stadium/)

In [31]:
df = pl.read_csv(DATA_DIR / "601.csv", dtypes={"visit_date": pl.Date})
print(df)

shape: (8, 3)
┌─────┬────────────┬────────┐
│ id  ┆ visit_date ┆ people │
│ --- ┆ ---        ┆ ---    │
│ i64 ┆ date       ┆ i64    │
╞═════╪════════════╪════════╡
│ 1   ┆ 2017-01-01 ┆ 10     │
│ 2   ┆ 2017-01-02 ┆ 109    │
│ 3   ┆ 2017-01-03 ┆ 150    │
│ 4   ┆ 2017-01-04 ┆ 99     │
│ 5   ┆ 2017-01-05 ┆ 145    │
│ 6   ┆ 2017-01-06 ┆ 1455   │
│ 7   ┆ 2017-01-07 ┆ 199    │
│ 8   ┆ 2017-01-09 ┆ 188    │
└─────┴────────────┴────────┘


MySQL
```mysql
with cte as (
    select *,
        lag(id, 2) over w as id1,
        lag(id, 1) over w as id2,
        lead(id, 1) over w as id4,
        lead(id, 2) over w as id5
    from Stadium
    where people >= 100
    window w as (order by id))
select id, visit_date, people
from cte
where
    (id = id4-1 and id = id5-2) or
    (id = id2+1 and id = id4-1) or
    (id1 = id-2 and id2 - id-1)
order by visit_date
```

[gaps and island](https://www.mssqltips.com/sqlservertutorial/9130/sql-server-window-functions-gaps-and-islands-problem/)

In [53]:
print(
    df.filter(col("people") >= 100)
    .sort("id")
    .filter(pl.count().over(col("id") - col("id").cum_count()) >= 3)
)

shape: (4, 3)
┌─────┬────────────┬────────┐
│ id  ┆ visit_date ┆ people │
│ --- ┆ ---        ┆ ---    │
│ i64 ┆ date       ┆ i64    │
╞═════╪════════════╪════════╡
│ 5   ┆ 2017-01-05 ┆ 145    │
│ 6   ┆ 2017-01-06 ┆ 1455   │
│ 7   ┆ 2017-01-07 ┆ 199    │
│ 8   ┆ 2017-01-09 ┆ 188    │
└─────┴────────────┴────────┘
